In [ ]:
!unzip "/content/drive/MyDrive/vision_transformer.zip" -d "/content/"

In [8]:
import os
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define the main dataset directory
dataset_dir = "/content/vision_transformer"

# Define the list of class names
class_names = ["Pothole", "speedbreaker"]

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through train, test, and valid subfolders
for split in ["train", "test", "valid"]:
    split_dir = os.path.join(dataset_dir, split)

    # Iterate through class subfolders (pothole and speedbreaker)
    for class_name in class_names:
        class_dir = os.path.join(split_dir, class_name)

        # Get a list of image files in the class folder
        image_files = glob.glob(os.path.join(class_dir, "*.jpg"))  # Adjust the file extension as needed

        # Load and preprocess each image, and assign a label
        for image_file in image_files:
            img = load_img(image_file, target_size=(224, 224))  # Adjust target size as needed
            img = img_to_array(img)
            img = img / 255.0  # Normalize pixel values
            images.append(img)

            label = class_names.index(class_name)
            labels.append(label)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# One-hot encode labels if needed
num_classes = len(class_names)
one_hot_labels = to_categorical(labels, num_classes=num_classes)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense, SeparableConvolution2D
from tensorflow.keras.models import Model

In [10]:
input_shape=(224,224,3)

In [11]:
from tensorflow.keras.layers import Input, SeparableConv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

def create_vgg16_new():
    inputs = Input(input_shape)

    # Block 1
    x = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    #x = Dropout(0.25)(x)

    # Block 3
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    #x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Fully Connected Layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # Output layer for classes (2 classes: Pothole and speedbreaker)
    outputs = Dense(2, activation='softmax')(x)

    # Creating the model
    model = Model(inputs, outputs)

    return model

In [12]:
vgg16_model = create_vgg16_new()

#Compile the model
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score

# Define the number of folds for cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

precision_scores = []
recall_scores = []
f1_scores = []

acc_scores =[]

# Perform k-fold cross-validation
for train_index, test_index in skf.split(images, labels):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = one_hot_labels[train_index], one_hot_labels[test_index]

    # Train the model
    vgg16_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

    # Make predictions
    y_pred = vgg16_model.predict(X_test)

    # Convert one-hot encoded labels back to class labels for evaluation
    y_true_classes = np.argmax(y_test, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Calculate precision, recall, and F1-score for this fold
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

    # Append the scores to the respective lists
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    acc_scores.append(accuracy)

# Calculate average precision, recall, and F1-score across all folds
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_accuracy = np.mean(acc_scores)

print("Average Accuracy : {:.2f}".format(avg_accuracy))
print("Average Precision: {:.2f}".format(avg_precision))
print("Average Recall: {:.2f}".format(avg_recall))
print("Average F1-score: {:.2f}".format(avg_f1))

Epoch 1/20
31/31 [==============================] - 42s 706ms/step - loss: 1.8441 - accuracy: 0.7711
Epoch 2/20
31/31 [==============================] - 21s 678ms/step - loss: 0.6931 - accuracy: 0.7918
Epoch 3/20
31/31 [==============================] - 20s 656ms/step - loss: 0.5775 - accuracy: 0.8278
Epoch 4/20
31/31 [==============================] - 20s 652ms/step - loss: 0.4546 - accuracy: 0.8670
Epoch 5/20
31/31 [==============================] - 20s 656ms/step - loss: 0.3469 - accuracy: 0.8856
Epoch 6/20
31/31 [==============================] - 21s 663ms/step - loss: 0.3998 - accuracy: 0.9000
Epoch 7/20
31/31 [==============================] - 21s 662ms/step - loss: 0.3786 - accuracy: 0.8814
Epoch 8/20
31/31 [==============================] - 20s 658ms/step - loss: 0.3552 - accuracy: 0.9082
Epoch 9/20
31/31 [==============================] - 20s 657ms/step - loss: 0.3007 - accuracy: 0.9072
Epoch 10/20
31/31 [==============================] - 20s 657ms/step - loss: 0.2098 - accura

ResourceExhaustedError: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))


num_folds = 5


metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
average_scores = [avg_accuracy, avg_precision, avg_recall, avg_f1]


plt.plot(metrics, average_scores, marker='o', color='b', label='Average Scores')
for i, metric_value in enumerate(average_scores):
    plt.text(metrics[i], metric_value, f'{metric_value:.2f}', ha='center', va='bottom')


for fold_idx in range(num_folds):
    fold_scores = [acc_scores[fold_idx], precision_scores[fold_idx], recall_scores[fold_idx], f1_scores[fold_idx]]
    plt.plot(metrics, fold_scores, marker='o', linestyle='--', label=f'Fold {fold_idx + 1}')
    for i, metric_value in enumerate(fold_scores):
        plt.text(metrics[i], metric_value, f'{metric_value:.2f}', ha='center', va='bottom')

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Metrics Across Folds')
plt.legend()
plt.grid(True)
plt.show()

In [13]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(vgg16_model, to_file='VGG16.png', show_shapes=True, dpi=300)

In [ ]:
vgg16_model.save("VGG16.h5")